## 查看GPU运行情况
若显存已被占用可在上方“代码执行程序”->“管理会话”里管理进程

In [0]:
!nvidia-smi

Sat Apr 27 04:20:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    19W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## 挂载Google Drive内的数据

In [1]:
from google.colab import drive
drive.mount('/Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /Drive


## 训练模型
若已有实现好的代码，亦可直接使用 ! python train.py 命令进行训练

In [0]:
from osgeo import gdal
import os
import random
import numpy as np
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.models import load_model

path = ('Drive/My Drive/湖北')
class_num = 11
bath_size = 15
steps_per_epoch = 15
epochs = 10

In [0]:
def TrainGenerator_v2(path,batchsize):
    files = os.listdir(path)
    length = files.__len__()
    while (True):
    
        sample = random.sample(range(length),batchsize)
        
        feature = []
        label = []

        for i in sample:
            dataset = gdal.Open(os.path.join(path,files[i]))
            data = dataset.ReadAsArray()
            dataset = None
            data[0]=data[0]%10
            data = np.delete(data,9,axis=0)
            feature.append(data[1:11,:,:])
            label.append(data[0,:,:])
        feature = np.array(feature)
        label = np.array(label)
        label[label>=11]=0
        feature = np.moveaxis(feature, 1, 3)
        label = to_categorical(label,class_num,dtype='uint8')


        yield feature,label

In [8]:
cd ..

/


In [9]:
myGene = TrainGenerator_v2(path=path,batchsize=bath_size)
model = load_model('Drive/My Drive/res_unet.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [11]:
model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])
model.fit_generator(myGene,epochs=epochs,steps_per_epoch=steps_per_epoch)
model.save('Drive/My Drive/res_unet.h5')

Epoch 1/10
15/15 [==============================] - 49s 3s/step - loss: 1.1910 - categorical_accuracy: 0.7889
Epoch 2/10
15/15 [==============================] - 31s 2s/step - loss: 0.9146 - categorical_accuracy: 0.7524
Epoch 3/10
15/15 [==============================] - 20s 1s/step - loss: 0.8742 - categorical_accuracy: 0.7295
Epoch 4/10
15/15 [==============================] - 20s 1s/step - loss: 0.7906 - categorical_accuracy: 0.7286
Epoch 5/10
15/15 [==============================] - 21s 1s/step - loss: 0.7081 - categorical_accuracy: 0.7572
Epoch 6/10
15/15 [==============================] - 20s 1s/step - loss: 0.7355 - categorical_accuracy: 0.7407
Epoch 7/10
15/15 [==============================] - 20s 1s/step - loss: 0.6625 - categorical_accuracy: 0.7602
Epoch 8/10
15/15 [==============================] - 20s 1s/step - loss: 0.7045 - categorical_accuracy: 0.7527
Epoch 9/10
15/15 [==============================] - 20s 1s/step - loss: 0.7087 - categorical_accuracy: 0.7529
Epoch 10/1